In [1]:
from requests import get
from bs4 import BeautifulSoup
import datetime
import regex as re
import pandas as pd
from unidecode import unidecode

In [18]:
log = []

In [22]:
def scraping(url,match,log):
    #get data from site
    print(f"Scraping for this match : {match}")
    response = get(url,headers= {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'} )

    #print status code

    print(response.status_code)

    if response.status_code == 200:
        pageSoup = BeautifulSoup(response.content, 'html.parser')
        time = []
        comment = []
        event = []
        event_player = []
        event_team = []
        comment_desc = []

        half_time_flag = False
        timer_flag = False
        comment_desc_dict = {True: "half time summary",
                        False : "full time summary"}

        counter = 0

        for x in pageSoup.findAll('div',class_ = "comment"):
            counter += 1

            if x.find_all("div", {"class": "comment-event"}):
                for q in x.findAll('div',class_ = "wrapper"):
                    for r in q.findAll('div',class_ = "event-icon"):
                        e = r.find("div")["class"][1]
                        e = e.replace("type-","")
                        e = unidecode(e)
                        

                    for r in q.findAll('div',class_ = "event-text has-additional"):
                        if r.find("div",{"class" : "event-text-main" }):
                            p1 = unidecode(r.find("div",{"class" : "event-text-main" }).text)
                        else:
                            p1 = ""
                        if r.find("div",{"class" : "event-text-additional" }):
                            p2 = unidecode(r.find("div",{"class" : "event-text-additional" }).text)
                            #e = x.find("div", {"class": "comment-event"}).find("event-text-additional").text
                        else:
                            p2 = ""
                        

                    for r in q.findAll('div',class_ = "event-crest"):
                        for d in r.findAll('a'):
                            #print(d)
                            te = unidecode(d.find("img")['alt'])
                            

                event_player.append([p1,p2])
                event.append(e)
                event_team.append(te)


            else:
                event.append("")
                event_player.append("")
                event_team.append("")
            for y in x.findAll('div',class_ = "comment-desc"):
                if y.find_all("span", {"class": "time"}):
                    timer_flag = True


                    t = y.find('span',class_ = "time").text
                    t = t.replace("'","")
                    #print(t)
                    time.append(t)
                    comment_desc.append("timer")
                else:
                    time.append("")
                    if time[-1] == "" and timer_flag:
                            half_time_flag = True
                    comment_desc.append(comment_desc_dict[half_time_flag])
                    #print("Summary")
                
                if y.find_all("span", {"class": "text"}):
                    c = y.find('span',class_ = "text").text
                    c = unidecode(c)
                    comment.append(c)
        

        print(f"Total scraped comments for this match: {counter} ")
        match_commentary_df = pd.DataFrame(list(zip(time, comment,event,event_player,event_team,comment_desc)),
                columns =['time', 'comment','event','event_player','event_team','comment_desc'])
        
        team_count = 0
        for teams in pageSoup.findAll('div',class_ = "widget-match-header__name"):
            if team_count == 0:
                match_commentary_df['home_team'] = unidecode(teams.find("span",{"class" : "widget-match-header__name--full" }).text)
                match_commentary_df['home_team_abbr'] = unidecode(teams.find("span",{"class" : "widget-match-header__name--short" }).text)
                team_count += 1
            elif team_count == 1:
                match_commentary_df['away_team'] = unidecode(teams.find("span",{"class" : "widget-match-header__name--full" }).text)
                match_commentary_df['away_team_abbr'] = unidecode(teams.find("span",{"class" : "widget-match-header__name--short" }).text)

        for score in pageSoup.findAll('div',class_ = "widget-match-header__score"):
            match_commentary_df['full_time_score'] = unidecode(score.find("span").text)
        

        return (match_commentary_df,1)
    else:
        log.append(match)
        return (0,0)


In [5]:
matchlinks = pd.read_csv("../../data/cl_data_21_22.csv")


In [6]:
matchlinks.head()

,match,match_day,live_ticker
0,Barcelona vs Bayern Muenchen,09/14/21,https://www.goal.com/en/match/barcelona-vs-bay...
1,Dynamo kyiv Vs Benfica,09/14/21,https://www.goal.com/en/match/dynamo-kyiv-vs-b...
2,Young boys Vs Manchester United,09/14/21,https://www.goal.com/en/match/young-boys-vs-ma...
3,Villarreal vs Atalanta,09/14/21,https://www.goal.com/en/match/villarreal-vs-at...
4,Sevilla vs Salzburg,09/14/21,https://www.goal.com/en/match/sevilla-vs-salzb...


In [23]:
commentory_df = pd.DataFrame(columns=['time', 'comment', 'event', 'event_player', 'event_team',
       'comment_desc', 'home_team', 'home_team_abbr', 'away_team',
       'away_team_abbr', 'full_time_score', 'match', 'date', 'link'])
counter = 0
for idx,row in matchlinks.iterrows():
    
    match = row['match']
    url = row['live_ticker']

    temp = scraping(url,match,log)
    if temp[1] == 1:
        counter += 1
        temp_df = temp[0]
        temp_df['match'] = match
        temp_df['date'] = row['match_day']
        temp_df['link'] = row['live_ticker']
        
        #print(temp_df)
        # print(temp_df.head(1))
        # print(temp_df.columns)

        commentory_df = pd.concat([commentory_df,temp_df])

print("Total matches scraped : {counter}")
    
    # if counter == 2:
    #     break

Scraping for this match : Barcelona vs Bayern Muenchen
200
Total scraped comments for this match: 78 
Scraping for this match : Dynamo kyiv Vs Benfica
200
Total scraped comments for this match: 81 
Scraping for this match : Young boys Vs Manchester United
200
Total scraped comments for this match: 89 
Scraping for this match : Villarreal vs Atalanta
200
Total scraped comments for this match: 82 
Scraping for this match : Sevilla vs Salzburg
200
Total scraped comments for this match: 83 
Scraping for this match : Lille vs Wolfsburg
200
Total scraped comments for this match: 83 
Scraping for this match : Chelsea vs Zenit
200
Total scraped comments for this match: 76 
Scraping for this match : Malmoe ff Vs Juventus
200
Total scraped comments for this match: 69 
Scraping for this match : Club brugge Vs Psg
200
Total scraped comments for this match: 76 
Scraping for this match : Manchester city Vs Rb Leipzig
200
Total scraped comments for this match: 94 
Scraping for this match : Atletico m

In [24]:
log

[]

In [25]:
commentory_df.to_csv("../../data/commentory.csv",index=False) 